# Multi-Agent System (AAI-520 Final Project)
---

Team Members (Group 4)
- Atul Aneja
- Juan Triana Martinez
- Mostafa Zamaniturk
- Ramesh Dhanasekaran
- Victoria Dorn

Code can be found on [github](https://github.com/vdorn5/aai-520-final-project/tree/main).

## Investment Research Agentic AI

This nobebook walks through the creation of an autonomous multi-agent system designed for performing dynamic, self-improving investment research using LLMs, financial data, and structured agent workflows. Built with CrewAI to orchestrate specialist agents in a collaborative research pipeline.

### How to run this notebook?

Clone the [github](https://github.com/vdorn5/aai-520-final-project/tree/main) repository and follow instructions on readme. The recommendation is to use the devcontainer setup, but alternatively you can install the `.devcontainer/requirements.txt` in your own setup. *Make sure to update the .env file with your OPEN_API_KEY, FRED_API_KEY, and NEWS_API_KEY.*

---
## Code Execution

In [ ]:
!pip install -r /content/requirements.txt > /dev/null

In [3]:
import os

# Create directories
os.makedirs("src/agents", exist_ok=True)
os.makedirs("src/tasks", exist_ok=True)
os.makedirs("src/tools", exist_ok=True)

In [4]:
%%writefile src/tools/yfinance_client.py
from crewai.tools import tool
import yfinance as yf
# --- Tool for Basic Stock Data ---
@tool("Stock Ticker Data Tool")
def get_stock_data(ticker: str) -> dict:
    """A tool to get basic financial data for a given stock ticker."""
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "longName": info.get("longName"),
        "sector": info.get("sector"),
        "industry": info.get("industry"),
        "marketCap": info.get("marketCap"),
        "trailingPE": info.get("trailingPE"),
        "forwardPE": info.get("forwardPE"),
        "dividendYield": info.get("dividendYield"),
        "fiftyTwoWeekHigh": info.get("fiftyTwoWeekHigh"),
        "fiftyTwoWeekLow": info.get("fiftyTwoWeekLow"),
        "regularMarketPrice": info.get("regularMarketPrice")
    }

# --- Tool for Financial Statements (for EarningsAnalyst) ---
@tool("Company Financial Statements Tool")
def get_financial_statements(ticker: str) -> str:
    """
    A tool to get the most recent annual financial statements (Income Statement,
    Balance Sheet, and Cash Flow) for a given stock ticker.
    """
    stock = yf.Ticker(ticker)

    # Fetch the most recent annual data
    income_statement = stock.income_stmt.iloc[:, 0]
    balance_sheet = stock.balance_sheet.iloc[:, 0]
    cash_flow = stock.cashflow.iloc[:, 0]

    # Format into a single string for the LLM
    report = f"""
    Income Statement:\n{income_statement.to_string()}\n
    Balance Sheet:\n{balance_sheet.to_string()}\n
    Cash Flow Statement:\n{cash_flow.to_string()}
    """
    return report

def get_yfinance_financials(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    return financials.to_string()

def get_analyst_recommendations(ticker):
    stock = yf.Ticker(ticker)
    return stock.recommendations.tail(3).to_string()

Overwriting src/tools/yfinance_client.py


In [5]:
%%writefile src/tools/news_analysis_tools.py
import os
import re
import torch
import nltk
from crewai.tools import tool
from newsapi import NewsApiClient
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# --- Class for Advanced News Analysis Pipeline (for NewsAnalyst) ---
class NewsAnalysisTools:
    @staticmethod
    @tool("Advanced News Analysis Tool")
    def news_analysis_pipeline(company_name: str) -> str:
        """
        A complete news analysis pipeline.
        1. Ingests news articles for a given company.
        2. Preprocesses the text of each article.
        3. Classifies the sentiment of each article using a financial model.
        4. Returns a consolidated list of processed articles for further analysis.
        """
        # Ensure NLTK data is available
        try:
            stopwords.words('english')
        except LookupError:
            nltk.download('stopwords', quiet=True)
            nltk.download('punkt', quiet=True)

        # Step 1: Ingest News
        newsapi = NewsApiClient(api_key=os.environ.get("NEWS_API_KEY"))
        try:
            top_headlines = newsapi.get_everything(
                q=company_name,
                language='en',
                sort_by='publishedAt',
                page_size=5
            )
            articles = top_headlines['articles']
        except Exception as e:
            return f"Error fetching news: {e}"

        processed_articles = []
        for article in articles:
            content = article['content'] or article['description'] or ""
            if not content:
                continue

            # Step 2: Preprocess Text
            preprocessed_text = NewsAnalysisTools._preprocess_text(content)

            # Step 3: Classify Sentiment
            sentiment = NewsAnalysisTools._classify_sentiment(preprocessed_text)

            processed_articles.append({
                "title": article['title'],
                "url": article['url'],
                "content_preview": preprocessed_text[:200] + "...",
                "sentiment": sentiment
            })

        return str(processed_articles)

    @staticmethod
    def _preprocess_text(text: str) -> str:
        """Internal method to clean and normalize text."""
        if not text:
            return ""
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = text.lower()
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(text)
        filtered_text = [w for w in word_tokens if not w in stop_words]
        return " ".join(filtered_text)

    @staticmethod
    def _classify_sentiment(text: str) -> str:
        """Internal method to classify sentiment using FinBERT."""
        if not text:
            return "Neutral"
        try:
            tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
            model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            with torch.no_grad():
                logits = model(**inputs).logits

            scores = {k: v for k, v in zip(model.config.id2label.values(), torch.softmax(logits, dim=0).tolist())}
            return max(scores, key=scores.get)
        except Exception as e:
            return f"Sentiment analysis failed: {e}"

Overwriting src/tools/news_analysis_tools.py


In [6]:
%%writefile src/tools/fred_client.py
from crewai.tools import tool
from fredapi import Fred

@tool("FRED Economic Data Tool")
def get_fred_data(series_id: str, limit: int = 5) -> str:
    """
    A tool to fetch economic data from the FRED API for a given series ID.
    Common series IDs include:
    - 'GDP': Gross Domestic Product
    - 'CPIAUCSL': Consumer Price Index for All Urban Consumers
    - 'UNRATE': Civilian Unemployment Rate
    """
    try:
        fred = Fred()
        data = fred.get_series(series_id).tail(limit)
        return data.to_string()
    except Exception as e:
        return f"Error fetching FRED data: {e}"

Overwriting src/tools/fred_client.py


In [7]:
%%writefile src/tools/memory_tools.py
import os
from datetime import datetime
from crewai.tools import tool

MEMORY_LOG_FILE = "memory_log.txt"

@tool("Read Memory Tool")
def read_memory(ticker: str) -> str:
    """A tool to read the memory log for any prior analysis on a given stock ticker."""
    try:
        with open(MEMORY_LOG_FILE, "r") as f:
            full_log = f.read()

        entries = full_log.split("\n---\n")
        relevant_memories = [entry for entry in entries if ticker.upper() in entry]

        if not relevant_memories:
            return f"No prior analysis found for {ticker}."

        return "\n---\n".join(relevant_memories)
    except FileNotFoundError:
        return "No memory log found. Starting fresh."

@tool("Save Memory Tool")
def save_memory(report: str, ticker: str) -> str:
    """A tool to save the final investment report to the memory log for future reference."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    memory_entry = f"Date: {timestamp} | Ticker: {ticker.upper()} | Report: {report}\n---\n"

    with open(MEMORY_LOG_FILE, "a") as f:
        f.write(memory_entry)

    return f"Report for {ticker.upper()} has been saved to memory."

Overwriting src/tools/memory_tools.py


In [8]:
%%writefile src/agents/earnings_analyst_agent.py
from crewai import Agent
from tools.yfinance_client import get_stock_data, get_financial_statements
from tools.memory_tools import read_memory

# Create an Earnings Analyst agent
earnings_analyst = Agent(
  role='Earnings Analyst',
  goal='Analyze the financial statements of a company to assess its financial health and performance, considering any past analyses.',
  backstory=(
    'A meticulous financial analyst with a deep understanding of corporate finance. '
    'You specialize in dissecting income statements, balance sheets, and cash flow statements '
    'to uncover underlying trends and financial stability. You also consult past reports to see what has changed.'
  ),
  tools=[get_stock_data, get_financial_statements, read_memory],
  verbose=True,
  allow_delegation=False
)

Overwriting src/agents/earnings_analyst_agent.py


In [9]:
%%writefile src/agents/news_analyst_agent.py
# NOTE: This file was created to satisfy the import in orchestrator.py
# The original project structure did not include this file.
from crewai import Agent
from tools.news_analysis_tools import NewsAnalysisTools

news_analyst = Agent(
    role='News Analyst',
    goal='Analyze news articles to gauge market sentiment for a given company.',
    backstory=(
        'An expert news analyst who scours the web for the latest articles, '
        'using advanced NLP techniques to determine the overall market sentiment. '
        'You are skilled at cutting through the noise to find what truly matters.'
    ),
    tools=[NewsAnalysisTools.news_analysis_pipeline],
    verbose=True,
    allow_delegation=False
)

Overwriting src/agents/news_analyst_agent.py


In [10]:
%%writefile src/agents/market_analyst_agent.py
# NOTE: This file was created to satisfy the import in orchestrator.py
# The original project structure did not include this file.
from crewai import Agent
from tools.fred_client import get_fred_data

market_analyst = Agent(
    role='Macroeconomic Market Analyst',
    goal='Analyze macroeconomic data to provide context for the stock market and a specific company.',
    backstory=(
        'A seasoned economist who specializes in tracking and interpreting macroeconomic indicators. '
        'You understand how factors like GDP growth, inflation, and unemployment impact financial markets '
        'and individual companies.'
    ),
    tools=[get_fred_data],
    verbose=True,
    allow_delegation=False
)

Overwriting src/agents/market_analyst_agent.py


In [11]:
%%writefile src/agents/investment_agents.py
# NOTE: This file was created to satisfy the import in orchestrator.py
# The original project structure did not include this file.
from crewai import Agent

investment_advisor = Agent(
    role='Investment Advisor',
    goal='Synthesize financial, news, and macroeconomic analyses to formulate a comprehensive investment recommendation.',
    backstory=(
        'A highly experienced investment advisor known for providing clear, actionable advice. '
        'You excel at integrating diverse data points into a cohesive investment thesis, '
        'balancing potential rewards with identified risks to guide clients.'
    ),
    verbose=True,
    allow_delegation=False
)

Overwriting src/agents/investment_agents.py


In [12]:
%%writefile src/agents/critic_agent.py
from crewai import Agent

critic_agent = Agent(
    role='Quality Assurance Critic',
    goal='Critique and refine the investment report to ensure it is comprehensive, accurate, and actionable.',
    backstory=(
        'A highly experienced investment strategist with a keen eye for detail. You are tasked with '
        'reviewing financial reports to identify any gaps, logical inconsistencies, or unsupported claims. '
        'Your feedback is crucial for producing a final, high-quality investment recommendation.'
    ),
    verbose=True,
    allow_delegation=False
)

Overwriting src/agents/critic_agent.py


In [13]:
%%writefile src/tasks/financial_tasks.py
from crewai import Task

memory_retrieval_task = Task(
    description='Retrieve any past analysis for the stock {ticker} from the memory log using the Read Memory Tool.',
    expected_output='A summary of past analysis for {ticker}, or a statement that no prior analysis was found.',
    agent=None
)

earnings_analysis_task = Task(
  description=(
    'Analyze the most recent annual financial statements for the stock {ticker}. '
    'First, consider any insights from prior analyses provided in the context. '
    'Then, examine the Income Statement, Balance Sheet, and Cash Flow Statement to identify '
    'key trends in revenue, profitability, debt, and cash flow. '
    'Provide a summary of the company\'s financial health.'
  ),
  expected_output=(
    'A concise summary of the company\'s financial health based on its latest '
    'annual financial statements, highlighting key trends and figures, and noting any changes from past analyses.'
  ),
  agent=None,
  context=[memory_retrieval_task]
)

news_analysis_task = Task(
    description=(
        'Conduct a comprehensive news analysis for the company associated with the ticker {ticker}. '
        'Use your advanced pipeline to determine the overall market sentiment. '
        'The final output should be a summary of the sentiment analysis.'
    ),
    expected_output=(
        'A summary of the recent news sentiment (positive, negative, or neutral) '
        'surrounding the company.'
    ),
    agent=None
)

market_analysis_task = Task(
    description=(
        'Analyze the current macroeconomic environment. Fetch the latest data for '
        'Gross Domestic Product (GDP) and the Consumer Price Index (CPIAUCSL). '
        'Summarize the current economic trends and their potential impact on the stock market '
        'and the company with ticker {ticker}.'
    ),
    expected_output=(
        'A summary of the current macroeconomic trends (e.g., inflation, economic growth) '
        'and a brief analysis of their potential impact on the company.'
    ),
    agent=None
)

advisory_draft_task = Task(
  description=(
    'Synthesize the financial statement analysis, news sentiment analysis, and macroeconomic '
    'context. Based on all this information, formulate a DRAFT investment recommendation '
    '(Buy, Hold, or Sell) for {ticker}. Provide a detailed justification for your recommendation, '
    'referencing specific data points from all three analyses.'
  ),
  expected_output=(
    'A draft investment report with a clear recommendation (Buy, Hold, or Sell) '
    'and a detailed rationale that integrates insights from the financial statements, '
    'news sentiment, and macroeconomic environment.'
  ),
  agent=None,
  context=[earnings_analysis_task, news_analysis_task, market_analysis_task]
)

report_critique_task = Task(
    description=(
        'Review the DRAFT investment report provided in the context. Your role is to act as a '
        'skeptical quality assurance analyst. Critique the report based on the following principles:\n'
        '1. Clarity and Cohesion: Is the main investment thesis clear and easy to understand?\n'
        '2. Evidentiary Support: Are all claims backed by specific data points from the analyses?\n'
        '3. Risk Assessment: Does the report adequately identify and discuss the primary risks?\n\n'
        'After your critique, produce a FINAL, polished investment report that incorporates your feedback '
        'and provides a definitive recommendation for {ticker}.'
    ),
    expected_output=(
        'A final, comprehensive, and polished investment report with a clear recommendation '
        '(Buy, Hold, or Sell) and a detailed, well-supported rationale that has been '
        'critically reviewed and refined.'
    ),
    agent=None,
    context=[advisory_draft_task]
)

Overwriting src/tasks/financial_tasks.py


In [41]:
%%writefile src/orchestrator.py
# main.py

import os
from dotenv import load_dotenv
from crewai import Crew, Process

# Load environment variables
load_dotenv()

# Import the tool we will call manually
from tools.memory_tools import save_memory

from agents.earnings_analyst_agent import earnings_analyst
from agents.news_analyst_agent import news_analyst
from agents.market_analyst_agent import market_analyst
from agents.critic_agent import critic_agent
from agents.investment_agents import investment_advisor

from tasks.financial_tasks import (
    memory_retrieval_task,
    earnings_analysis_task,
    news_analysis_task,
    market_analysis_task,
    advisory_draft_task,
    report_critique_task
)



# Assign agents to their respective tasks
memory_retrieval_task.agent = earnings_analyst
earnings_analysis_task.agent = earnings_analyst
news_analysis_task.agent = news_analyst
market_analysis_task.agent = market_analyst
advisory_draft_task.agent = investment_advisor
report_critique_task.agent = critic_agent

# Define the context for the earnings analysis task
earnings_analysis_task.context = [memory_retrieval_task]

def run_analysis(ticker: str):
    """
    Initializes and runs the financial analysis crew for a given stock ticker.
    """
    financial_crew = Crew(
      agents=[earnings_analyst, news_analyst, market_analyst, investment_advisor, critic_agent],
      tasks=[
          memory_retrieval_task,
          earnings_analysis_task,
          news_analysis_task,
          market_analysis_task,
          advisory_draft_task,
          report_critique_task
      ],
      process=Process.sequential,
      verbose=True
    )

    # Kick off the crew's work
    final_report = financial_crew.kickoff(inputs={'ticker': ticker})

    print("\n\n########################")
    print("## Final Analysis Report:")
    print("########################")
    print(final_report)

    # Manually save the final report to the memory log
    print("\n\n########################")
    print("## Saving Report to Memory...")
    print("########################")
    save_status = save_memory.run(report=final_report, ticker=ticker)
    print(save_status)

if __name__ == "__main__":
    stock_ticker = "TSLA"
    print(f"🚀 Starting analysis for {stock_ticker}...")
    run_analysis(stock_ticker)

Overwriting src/orchestrator.py

In [43]:
%run src/orchestrator.py

🚀 Starting analysis for TSLA...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d8006aaf-4e5b-4768-aed6-319a9001bdb4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Earnings Analyst                                                                                        │
│                                                                                                                 │
│  Task: Retrieve any past analysis for the stock TSLA from the memory log using the Read Memory Tool.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Earnings Analyst                                                                                        │
│                                                                                                                 │
│  Thought: I need to check the memory log for any prior analysis on the stock TSLA.                              │
│                                                                                                                 │
│  Using Tool: Read Memory Tool                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Assurance Critic                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Polished Investment Recommendation Report for TSLA**                                                         │
│  **Recommendation: Buy**                                                                                        │
│                                                                                                                 │
│  ### Executive Summary:                                                                                         │
│  This investment report evaluates Tesla Inc. (TSLA) through a thorough financial analysis, market sentiment     │
│  overview, and macroeconomic context. The conclusion suggests a** Buy** recommendation due to TSLA's robust     │
│  financial performance, strategic market positioning in the electric vehicle (EV) sector, and capacity for      │
│  future growth amidst evolving economic conditions.                                                             │
│                                                                                                                 │
│  ### 1. Financial Summary Analysis:                                                                             │
│                                                                                                                 │
│  - **Income Statement Metrics:**                                                                                │
│  TSLA reported total revenue of **$97.69 billion** for the last fiscal year, with a net income of **$7.13       │
│  billion**, yielding a notable net profit margin of **approximately 7.28%**. This reflects the company’s        │
│  adeptness at cost management and strong sales trajectory amid the competitive automotive landscape. The        │
│  reported EBITDA stands at **$14.70 billion**, showcasing operational efficiency, further solidifying TSLA's    │
│  financial health.                                                                                              │
│                                                                                                                 │
│  - **Balance Sheet Strength:**                                                                                  │
│  A detailed analysis of TSLA’s balance sheet reveals substantial equity of **$73.91 billion** against total     │
│  liabilities of **$48.39 billion**. The resulting equity-to-debt ratio of **1.53** and a low debt-to-equity     │
│  ratio of **0.18** illustrate a conservative leverage strategy. This strong balance sheet indicates resilience  │
│  and a robust safety net for the company as it faces market fluctuations.                                       │
│                                                                                                                 │
│  - **Cash Flow Dynamics:**                                                                                      │
│  TSLA’s operating cash flow is substantial at **$14.92 billion**, signaling strong cash generation capability,  │
│  while free cash flow stands at **$3.58 billion**. This financial flexibility supports ongoing investments and  │
│  potential shareholder returns. However, it is critical to note the significant capital expenditures of         │
│  **$11.34 billion**, which underscore the company’s robust commitment to growth through reinvestments needed    │
│  to sustain future revenues.                           

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b21b612e-8725-4285-b07a-cf95057af862                                                                     │
│  Agent: Quality Assurance Critic                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d8006aaf-4e5b-4768-aed6-319a9001bdb4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Polished Investment Recommendation Report for TSLA**                                           │
│  **Recommendation: Buy**                                                                                        │
│                                                                                                                 │
│  ### Executive Summary:                                                                                         │
│  This investment report evaluates Tesla Inc. (TSLA) through a thorough financial analysis, market sentiment     │
│  overview, and macroeconomic context. The conclusion suggests a** Buy** recommendation due to TSLA's robust     │
│  financial performance, strategic market positioning in the electric vehicle (EV) sector, and capacity for      │
│  future growth amidst evolving economic conditions.                                                             │
│                                                                                                                 │
│  ### 1. Financial Summary Analysis:                                                                             │
│                                                                                                                 │
│  - **Income Statement Metrics:**                                                                                │
│  TSLA reported total revenue of **$97.69 billion** for the last fiscal year, with a net income of **$7.13       │
│  billion**, yielding a notable net profit margin of **approximately 7.28%**. This reflects the company’s        │
│  adeptness at cost management and strong sales trajectory amid the competitive automotive landscape. The        │
│  reported EBITDA stands at **$14.70 billion**, showcasing operational efficiency, further solidifying TSLA's    │
│  financial health.                                                                                              │
│                                                                                                                 │
│  - **Balance Sheet Strength:**                                                                                  │
│  A detailed analysis of TSLA’s balance sheet reveals substantial equity of **$73.91 billion** against total     │
│  liabilities of **$48.39 billion**. The resulting equity-to-debt ratio of **1.53** and a low debt-to-equity     │
│  ratio of **0.18** illustrate a conservative leverage strategy. This strong balance sheet indicates resilience  │
│  and a robust safety net for the company as it faces market fluctuations.                                       │
│                                                                                                                 │
│  - **Cash Flow Dynamics:**                                                                                      │
│  TSLA’s operating cash flow is substantial at **$14.92 billion**, signaling strong cash generation capability,  │
│  while free cash flow stands at **$3.58 billion**. This financial flexibility supports ongoing investments and  │
│  potential shareholder returns. However, it is critical to note the significant capital expenditures of         │
│  **$11.34 billion**, which underscore the company’s ro

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): N


########################
## Final Analysis Report:
########################
**Polished Investment Recommendation Report for TSLA**  
**Recommendation: Buy**

### Executive Summary:
This investment report evaluates Tesla Inc. (TSLA) through a thorough financial analysis, market sentiment overview, and macroeconomic context. The conclusion suggests a** Buy** recommendation due to TSLA's robust financial performance, strategic market positioning in the electric vehicle (EV) sector, and capacity for future growth amidst evolving economic conditions.

### 1. Financial Summary Analysis:

- **Income Statement Metrics:**  
TSLA reported total revenue of **$97.69 billion** for the last fiscal year, with a net income of **$7.13 billion**, yielding a notable net profit margin of **approximately 7.28%**. This reflects the company’s adeptness at cost management and strong sales trajectory amid the competitive automotive landscap

## Summary:

The project effectively demonstrates multi-agent collaboration through defined roles, goals, and a sequential workflow. This project demonstrates a multi-agent system for investment research using CrewAI. Key aspects include:

**Agent Design and Workflows:**
- Specialized agents (Earnings Analyst, News Analyst, Market Analyst, Investment Advisor, Critic) with distinct roles and tasks/goals.
- A sequential workflow orchestrated by CrewAI, where tasks are linked and the output of one task feeds into the next, ensuring a structured research pipeline.

**Agent Functions and Capabilities:**
- Each agent is equipped with specific tools (e.g., for fetching financial data, performing news sentiment analysis, retrieving macroeconomic data) that define their capabilities and allow them to interact with external data sources.

**Evaluation and Iteration:**
- The project incorporates elements for evaluation and iteration, such as defined expected outputs for tasks, a memory tool to store and retrieve past analyses, and a critic agent for reviewing and refining the final report. This lays the groundwork for continuous improvement of the analysis process.